# load the AGORA2 models and media

In [2]:
from modelseedpy.biochem import from_local

msdb = from_local("../../ModelSEEDDatabase/")
print(dir(msdb.compounds[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_annotation', '_bound', '_id', '_model', '_reaction', '_repr_html_', '_set_id_with_model', 'abbr', 'annotation', 'charge', 'compartment', 'constraint', 'copy', 'delta_g', 'delta_g_error', 'elements', 'flags', 'formula', 'formula_weight', 'id', 'inchi', 'inchi_key', 'is_abstract', 'is_cofactor', 'is_core', 'is_obsolete', 'linked_compound', 'mass', 'model', 'name', 'names', 'notes', 'pka', 'pkb', 'reactions', 'remove_from_model', 'seed_id', 'shadow_price', 'smiles', 'source', 'summary', 'to_template_compartment_compound', 'y']


In [3]:
from cobra.io import read_sbml_model
from glob import glob

def get_model(path):
    try:  model = read_sbml_model(path)
    except:  return

# models = []
# for model in glob("../../AGORA2_models/*.xml"):
#     try:     models.append(read_sbml_model(model))
#     except:  print(model, " is broken")

from multiprocess import Pool
pool = Pool(24)
args = glob("../../AGORA2_models/*.xml")
print(args[0])
pool.map(get_model, args)

../../AGORA2_models/Klebsiella_aerogenes_CAV1320.xml


# simulate the community

In [ ]:
%run ../../ModelSEEDpy/modelseedpy/community/MSCommunity.py


# 

In [ ]:
ws_client = util.ws_client()
output = ws_client.list_objects({"ids":[178418]})
genome_hash = {}
for item in output:
    genome_hash[item[1]] = item
asvssets_with_missing_genomes = {}
missing_genomes = {}
asvset_genomes = util.load("asvset_genomes")
for asvset in asvset_genomes:
    for genomeid in asvset_genomes[asvset]:
        if genomeid not in genome_hash and genomeid+".RAST" not in genome_hash:
            if genomeid not in missing_genomes:
                missing_genomes[genomeid] = []
            if asvset not in missing_genomes[genomeid]:
                missing_genomes[genomeid].append(asvset)
            if asvset not in asvssets_with_missing_genomes:
                asvssets_with_missing_genomes[asvset] = []
            if genomeid not in asvssets_with_missing_genomes[asvset]:
                asvssets_with_missing_genomes[asvset].append(genomeid)
util.save("missing_genomes", missing_genomes)
util.save("asvssets_with_missing_genomes", asvssets_with_missing_genomes)
for genomeid in missing_genomes:
    print(genomeid,len(missing_genomes[genomeid]))
for asvset in asvssets_with_missing_genomes:
    print(asvset,len(asvssets_with_missing_genomes[asvset]),len(asvset_genomes[asvset]))
    if len(asvset_genomes[asvset]) <= 2:
        for genomeid in asvset_genomes[asvset]:
            if genomeid in missing_genomes:
                print(genomeid)